### Регрессия Надарая-Ватсона

In [115]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
from sklearn.base import BaseEstimator
from scipy.spatial.distance import cdist
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

In [116]:
X, y = load_boston(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.5, shuffle=True)

In [117]:
house_data=load_boston()
df=pd.DataFrame(pd.DataFrame(house_data.data, columns=house_data.feature_names))
df.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33


In [118]:
def K(distance,h=1):
     ret = np.array(distance) / h
     return np.exp(-ret ** 2)

In [119]:
class NV(BaseEstimator):
    def __init__(self, h=1):
        self.X, self.Y = None, None
        self.h=h
        self.kernel = lambda x: K(x, self.h)

    def predict(self, X):
        features = np.sum(self.Y * self.kernel(cdist(X, self.X)), axis=-1)
        return features / (np.sum(self.kernel(cdist(X, self.X)), axis=-1) + 1e-10)

    def fit(self, X, Y):
        self.X, self.Y = np.array(X), np.array(Y)

In [120]:
model = NV(h=100)
model.fit(X_train, y_train)
mean_absolute_error(model.predict(X_test), y_test)

6.03357562477173

In [121]:
grid_search = GridSearchCV(model, {'h': list(range(10, 1000))}, scoring='r2')
grid_search.fit(X_train, y_train)
grid_search.best_params_

{'h': 15}

In [122]:
mean_absolute_error(grid_search.predict(X_test), y_test)

4.736282206335681